In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = "{:.2f}".format

df = pd.read_csv('../data/data.csv')
df

,treat,age,educ,race,married,nodegree,re74,re75,re78
0,1,37,11,black,1,1,0.00,0.00,9930.05
1,1,22,9,hispan,0,1,0.00,0.00,3595.89
2,1,30,12,black,0,0,0.00,0.00,24909.45
3,1,27,11,black,0,1,0.00,0.00,7506.15
4,1,33,8,black,0,1,0.00,0.00,289.79
...,...,...,...,...,...,...,...,...,...
609,0,18,11,white,0,1,0.00,0.00,10150.50
610,0,24,1,hispan,1,1,0.00,0.00,19464.61
611,0,21,18,white,0,0,0.00,0.00,0.00
612,0,32,5,black,1,1,0.00,0.00,187.67


In [2]:
X = df[['age', 'educ', 'race', 'married', 'nodegree']]
t = df['treat']
y = df['re78']

## Coarsen data

In [3]:
X_coarse = X.copy()
X_coarse['age'] = pd.cut(X_coarse['age'], 5)
X_coarse['educ'] = pd.cut(X_coarse['educ'], 5)
X_coarse['race'] = X_coarse['race'].replace({'black': 'not white', 'hispan': 'not white'})
X_coarse

,age,educ,race,married,nodegree
0,"(31.6, 39.4]","(10.8, 14.4]",not white,1,1
1,"(15.961, 23.8]","(7.2, 10.8]",not white,0,1
2,"(23.8, 31.6]","(10.8, 14.4]",not white,0,0
3,"(23.8, 31.6]","(10.8, 14.4]",not white,0,1
4,"(31.6, 39.4]","(7.2, 10.8]",not white,0,1
...,...,...,...,...,...
609,"(15.961, 23.8]","(10.8, 14.4]",white,0,1
610,"(23.8, 31.6]","(-0.018, 3.6]",not white,1,1
611,"(15.961, 23.8]","(14.4, 18.0]",white,0,0
612,"(31.6, 39.4]","(3.6, 7.2]",not white,1,1


## Perform exact matching

In [4]:
# Add concatenated variable capturing the combination
X_coarse.join(t).groupby(['age', 'educ', 'race', 'married', 'nodegree'])['treat'].nunique().sort_values()==2

age             educ           race       married  nodegree
(15.961, 23.8]  (-0.018, 3.6]  not white  0        0           False
(31.6, 39.4]    (7.2, 10.8]    white      1        0           False
                (14.4, 18.0]   not white  0        1           False
                                          1        0           False
                                                   1           False
                                                               ...  
                (10.8, 14.4]   not white  1        0            True
(15.961, 23.8]  (7.2, 10.8]    not white  1        1            True
(47.2, 55.0]    (3.6, 7.2]     not white  0        1            True
(31.6, 39.4]    (7.2, 10.8]    white      0        1            True
                               not white  1        1            True
Name: treat, Length: 200, dtype: bool

In [5]:
# Add weight
# Add function that calculates weights depending on estimand

## Analyse causal impact

In [6]:
# Weighted mean